In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/03/02 23:08:07 WARN Utils: Your hostname, lat resolves to a loopback address: 127.0.1.1; using 192.168.1.14 instead (on interface wlp2s0)
22/03/02 23:08:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/02 23:08:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!wc -l fhvhv_tripdata_2021-02.csv

11613943 fhvhv_tripdata_2021-02.csv


In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-02.csv')

In [5]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [6]:
!head -n 1001 fhvhv_tripdata_2021-02.csv > head.csv

In [7]:
import pandas as pd

In [8]:
df_pandas = pd.read_csv('head.csv')

In [9]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [10]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [11]:
from pyspark.sql import types

In [12]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [13]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [14]:
df = df.repartition(24)


In [15]:
df.write.parquet('fhvhv/2021/02/')

22/03/02 23:10:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/02 23:10:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/02 23:10:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


Q3. How many taxi trips were there on February 15?

In [16]:
from pyspark.sql.functions import dayofmonth
df.filter(dayofmonth(df.pickup_datetime) == 15).count()

367170

Q4. Trip starting on which day was the longest?

In [17]:
df.orderBy(-(df.dropoff_datetime - df.pickup_datetime)).first().pickup_datetime

datetime.datetime(2021, 2, 11, 13, 40, 44)

Q5. Stages for most frequent dispatching_base_num?

In [22]:
rdd = df.rdd
rdd.cache()

MapPartitionsRDD[45] at javaToPython at NativeMethodAccessorImpl.java:0

In [34]:

rdd.map(lambda r: (r.dispatching_base_num, 1)).reduceByKey(lambda a, b: a + b).takeOrdered(1, key = lambda a: -a[1])

[('B02510', 3233664)]

Q6. Most common locations pair?


In [39]:
df_zones_rdd = spark.read.parquet('/home/adi/Downloads/nytaxi/taxi_zone_lookup.parquet').rdd

In [45]:
commonLocationPair = rdd.map(lambda r: ((r.PULocationID,r.DOLocationID), 1)).reduceByKey(lambda a, b: a + b).takeOrdered(1, key = lambda a: -a[1])[0][0]

In [46]:
commonLocationPair

(76, 76)

In [55]:
df_zones_rdd.filter(lambda r: r[0] == commonLocationPair[0]).first()[2]+"/"

'East New York'